!pip install transformers

In [11]:
!pip install transformers

     |████████████████████████████████| 573kB 3.3MB/s 
     |████████████████████████████████| 890kB 40.5MB/s 
     |████████████████████████████████| 1.0MB 48.1MB/s 
     |████████████████████████████████| 3.7MB 33.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.41-cp36-none-any.whl size=893334 sha256=6b1e49c8f540927c66777eaae19fdf67d2703b635747594d11ea23a76d17ff7f
  Stored in directory: /root/.cache/pip/wheels/22/5a/d4/b020a81249de7dc63758a34222feaa668dbe8ebfe9170cc9b1
Successfully built sacremoses


In [12]:
import pandas as pd

print('Parsing the dataset .tsv file...')
comments = pd.read_csv('/content/drive/My Drive/Deep Learning Data/Annotated Attack Comment/attack_annotated_comments.tsv', sep = '\t', index_col = 0)
annotations = pd.read_csv('/content/drive/My Drive/Deep Learning Data/Annotated Attack Comment/attack_annotations.tsv', sep = '\t')

print('    Done.')

Parsing the dataset .tsv file...
    Done.


In [13]:
# Display the first five rows of the table.
comments.head()

,comment,year,logged_in,ns,sample,split
rev_id,,,,,,
37675,`-NEWLINE_TOKENThis is not ``creative``. Thos...,2002,False,article,random,train
44816,`NEWLINE_TOKENNEWLINE_TOKEN:: the term ``stand...,2002,False,article,random,train
49851,"NEWLINE_TOKENNEWLINE_TOKENTrue or false, the s...",2002,False,article,random,train
89320,"Next, maybe you could work on being less cond...",2002,True,article,random,dev
93890,This page will need disambiguation.,2002,True,article,random,train


In [14]:
comments[['comment', 'split']].groupby('split').count()

,comment
split,
dev,23160
test,23178
train,69526


In [15]:
annotations.head()

,rev_id,worker_id,quoting_attack,recipient_attack,third_party_attack,other_attack,attack
0,37675,1362,0.0,0.0,0.0,0.0,0.0
1,37675,2408,0.0,0.0,0.0,0.0,0.0
2,37675,1493,0.0,0.0,0.0,0.0,0.0
3,37675,1439,0.0,0.0,0.0,0.0,0.0
4,37675,170,0.0,0.0,0.0,0.0,0.0


We have multiple labels and each comment have multiple people ratings

In [0]:
for att_type in ['quoting_attack', 'recipient_attack', 'third_party_attack', 'other_attack', 'attack']:

    labels = annotations.groupby('rev_id')[att_type].mean() > 0.5

    # Join labels and comments
    comments[att_type] = labels




In [17]:
comments.head()

,comment,year,logged_in,ns,sample,split,quoting_attack,recipient_attack,third_party_attack,other_attack,attack
rev_id,,,,,,,,,,,
37675,`-NEWLINE_TOKENThis is not ``creative``. Thos...,2002,False,article,random,train,False,False,False,False,False
44816,`NEWLINE_TOKENNEWLINE_TOKEN:: the term ``stand...,2002,False,article,random,train,False,False,False,False,False
49851,"NEWLINE_TOKENNEWLINE_TOKENTrue or false, the s...",2002,False,article,random,train,False,False,False,False,False
89320,"Next, maybe you could work on being less cond...",2002,True,article,random,dev,False,False,False,False,False
93890,This page will need disambiguation.,2002,True,article,random,train,False,False,False,False,False


In [0]:
# Remove newline and tab tokens
comments['comment'] = comments['comment'].apply(lambda x: x.replace("NEWLINE_TOKEN", ""))
comments['comment'] = comments['comment'].apply(lambda x: x.replace("TAB_TOKEN", ""))

In [19]:
comments.head()

,comment,year,logged_in,ns,sample,split,quoting_attack,recipient_attack,third_party_attack,other_attack,attack
rev_id,,,,,,,,,,,
37675,`-This is not ``creative``. Those are the dic...,2002,False,article,random,train,False,False,False,False,False
44816,`:: the term ``standard model`` is itself less...,2002,False,article,random,train,False,False,False,False,False
49851,"True or false, the situation as of March 2002 ...",2002,False,article,random,train,False,False,False,False,False
89320,"Next, maybe you could work on being less cond...",2002,True,article,random,dev,False,False,False,False,False
93890,This page will need disambiguation.,2002,True,article,random,train,False,False,False,False,False


In [20]:
comments.iloc[:, 6:].to_numpy().astype(int)

array([[0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       ...,
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0]])

In [0]:
# fit a simple text classifier
train_comments = comments.query("split=='train'")
dev_comments = comments.query("split == 'dev'")
test_comments = comments.query("split=='test'")

In [26]:
### lets see a couple of example 
import textwrap
import random

# Wrap text to 80 characters.
wrapper = textwrap.TextWrapper(width=80) 

# Filter to just the "attack" comments.
attack_examples = train_comments.query('attack')['comment']

# Randomly choose some examples.
for i in range(10):
    j = random.choice(attack_examples.index)
    
    print('')
    print(wrapper.fill(attack_examples[j]))
    print('')


 The fucking mother-fucker won't fucking leave me fucking alone! Please fucking
make him fucking stop!


Psycho ==Why do you keep telling us that our page is being vandalized!!!??? It
is our page! Can we do what we want? Nobody likes you! You are such a dickhead!
You lesbian butch!~~Colleen&VanessaNEWLINE;_TOKEN==


Why are you doing this? Are you some sort of sadist or troll?


i know you are racist, just admit it


To Kralizec you are a unintelligent, dull, brainless, obtuse, witless,foolish,
fatuous, inane, nonsensical, silly,daft, futile, ludicrous, ridiculous,
laughable, senseless, absurd, asinine,unwise, senseless, ill-advised, imprudent
(formal), injudicious, thoughtless, rash, irresponsible, reckless, heedless,
fugly, ugly, cunt/dick head, mean,shit licking, wanker,  son of a bitch.Go ahead
block me for another 7 days you ass hole.p.s the rank of administrator is an
affront to freedom of expression, you and the rest of your fucking
tyrant/dictator friends oppress the rest of w

In [0]:
### helper functions take dataframe  and return (input_ids, attention_ids, labels) from BertTokenizer:
from transformers import BertConfig, BertModel, BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case = True)

In [0]:
import torch
import numpy as np

def return_tokens(df,pretrain_tokenizer, max_length = 128):
    """Args:
            :df (dataframe): take an input dataframe similar to 'comments' dataframe
            :max_length (int): maximum length of the comment, must be less than 512 to fit into bert tokenizer
            :return_attention_mask (bool)
            :padding: padding comments that shorter than max_length"""

    df = df 

    tokenizer = pretrain_tokenizer

    input_ids = []
    attention_ids = []

    for sent in df['comment']:
        encoded_sent = tokenizer.encode_plus(sent, 
                                            add_special_tokens = True, # add[CLS] and [SEP] to the beginning and end of sentences, specially used for classification
                                            max_length = max_length,
                                            pad_to_max_length = True, #pad to max length
                                            return_attention_mask = True,
                                            return_tensor ='pt' # return pytorch tensor type
                                            )
        ids = torch.from_numpy(np.array(encoded_sent['input_ids'])).unsqueeze(0)
        attention = torch.from_numpy(np.array(encoded_sent['attention_mask'])).unsqueeze(0)


        input_ids.append(ids)
        attention_ids.append(attention)

    #input and attention tensors
    input_tensor = torch.cat(input_ids)
    attention_tensor = torch.cat(attention_ids)

    #label tensor
    labels = df.iloc[:, 6:].to_numpy().astype(int)
    label_tensor = torch.from_numpy(labels).type(torch.float32)


    return input_tensor, attention_tensor, label_tensor

In [0]:
### Getting these tensor ready
train_input, train_attention, train_label = return_tokens(train_comments, tokenizer)

test_input, test_attention, test_label = return_tokens(test_comments, tokenizer)

dev_input, dev_attention, dev_label = return_tokens(dev_comments, tokenizer)

In [30]:
!pip install pytorch-ignite

     |████████████████████████████████| 112kB 3.5MB/s 


In [0]:
from torch.utils.data import TensorDataset, DataLoader

In [0]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [0]:
## Creating dataloader for train, val, and test sets
train_set = TensorDataset(train_input, train_attention, train_label)
train_loader = DataLoader(train_set, batch_size = 16, num_workers = 1)

test_set = TensorDataset(test_input, test_attention, test_label)
test_loader = DataLoader(test_set, batch_size = 16, num_workers = 1)

val_set = TensorDataset(dev_input, dev_attention, dev_label)
val_loader = DataLoader(val_set, batch_size = 16, num_workers = 1)

In [34]:
### Create BertModel with classification head
config = BertConfig.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

In [0]:
import os
from ignite.engine import Engine, Events
from ignite.metrics import RunningAverage, Accuracy
from ignite.handlers import ModelCheckpoint
from ignite.contrib.handlers import CosineAnnealingScheduler, PiecewiseLinear, create_lr_scheduler_with_warmup, ProgressBar

In [2]:
## Model bert + multi_class classification heads
import torch.nn as nn
import torch.nn.functional as F


class Multiclass_bert(nn.Module):
    def __init__(self, hidden_dim, pretrained_model, num_classes):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.bert = pretrained_model
        self.num_classes = num_classes

        self.classification_head = nn.Linear(self.hidden_dim, self.num_classes)

    def forward(self, input_ids, labels, attention_mask):

        hidden_states, cls = self.bert(input_ids, 
                                  token_type_ids = None, 
                                  attention_mask = attention_mask) # return last hidden state size of (batch_size, max_length, hidden_dim)

        sum_hidden = torch.sum(hidden_states, dim =1) # return tensor of (batch_size, hidden_dim)

        hiddens = self.classification_head(sum_hidden) #return (batch_size, num_classes)

        clf_logits = torch.sigmoid(hiddens)

        ones = torch.ones((clf_logits.shape))
        zeros = torch.zeros((clf_logits.shape))

        logits = torch.where(clf_logits > 0.5, ones , zeros)
        
        
        loss_fct = nn.BCEWithLogitsLoss()
        loss = loss_fct(hiddens, labels)

        return hiddens, logits,  loss

model = Multiclass_bert(config.hidden_dim,bert_model,5)
model = model.to(device)

NameError: ignored

In [0]:
from collections import namedtuple

AdaptationConfig = namedtuple('AdaptationConfig',
  field_names="adapters_dim, num_classes, dropout, initializer_range, batch_size, lr, max_norm, n_epochs,"
              "n_warmup, valid_set_prop, gradient_accumulation_steps, device,"
              "log_dir, dataset_cache")
adapt_args = AdaptationConfig(
               32         , 6          , 0.1    , 0.02             , 16        , 6.5e-4, 1.0   , 3,
               10      , 0.1           , 1, "cuda" if torch.cuda.is_available() else "cpu",
               "./"   , "./dataset_cache.bin")

In [0]:
## training loop
optimizer = torch.optim.Adam(model.parameters(), lr=adapt_args.lr)

##training function and trainer
def update(engine, batch):

    # Activate 'train' mode for model
    model.train()
    #zero optimizer's gradient
    optimizer.zero_grad()

    #get inputs from batch
    input_ids = batch[0].to(device) ## size of (batch_size, sequence_length, hiddin_dims)

    attention_mask = batch[1].to(device)## size of (batch_size, sequence_length, hiddin_dims)

    labels = batch[2].to(device) ## Size of (batch_size, num_classes)

    #model outputs
    _, _, loss = model(input_ids, 
                          attention_mask = attention_mask,
                          labels = labels)
    
    loss = loss/adapt_args.gradient_accumulation_steps
    if engine.state.iteration % adapt_args.gradient_accumulation_steps == 0:
        optimizer.step()
        optimizer.zero_grad()
    return loss.item()
    
    #backward
    loss.backward()
    
    #next step
    optimizer.step()
    
    #return value of loss as float
    return loss.item()

trainer = Engine(update)

In [0]:
def inference(engine, batch):

    #evaluation mode of model
    model.eval()
    
    with torch.no_grad():# no gradient

        ## get inputs from btach
        input_ids = batch[0].to(device)
        
        attention_mask = batch[1].to(device)
        
        labels = batch[2].to(device)


    # output_model
    _, output,  loss = model(input_ids,
                          attention_mask = attention_mask,
                          labels = labels)
    
    return output, labels
    
    
evaluator = Engine(inference)

In [0]:
# Learning rate schedule: linearly warm-up to lr and then to zero
scheduler = PiecewiseLinear(optimizer, 'lr', [(0, 0.0), (adapt_args.n_warmup, adapt_args.lr),
                                              (len(train_loader)* adapt_args.n_epochs, 0.0)])
trainer.add_event_handler(Events.ITERATION_STARTED, scheduler)


# Attache metric to evaluator & evaluation to trainer: evaluate on valid set after each epoch
Accuracy().attach(evaluator, "accuracy")
@trainer.on(Events.EPOCH_COMPLETED)
def log_validation_results(engine):
    evaluator.run(val_loader)
    print(f"Validation Epoch: {engine.state.epoch} Error rate: {100*(1 - evaluator.state.metrics['accuracy'])}")

#Add progressbar with loss
RunningAverage(output_transform=lambda x: x).attach(trainer, "loss")
ProgressBar(persist=True).attach(trainer, metric_names=['loss'])

In [218]:
# Save checkpoints and finetuning config
checkpoint_handler = ModelCheckpoint(adapt_args.log_dir, 'finetuning_checkpoint', save_interval=1, require_empty=False)
trainer.add_event_handler(Events.EPOCH_COMPLETED, checkpoint_handler, {'mymodel': model})
torch.save(adapt_args, os.path.join(adapt_args.log_dir, 'fine_tuning_args.bin'))

/usr/local/lib/python3.6/dist-packages/ignite/handlers/checkpoint.py:369: UserWarning: Argument save_interval is deprecated and should be None. Please, use events filtering instead, e.g. Events.ITERATION_STARTED(every=1000)
  warnings.warn(msg)


In [219]:
trainer.run(train_loader, max_epochs=adapt_args.n_epochs)

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1351: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Current run is terminating due to exception: .
Engine run is terminating due to exception: .


KeyboardInterrupt: ignored